<a href="https://colab.research.google.com/github/smypmsa/pdf-to-table/blob/main/OCR_ed_PDF_to_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 PDF Table Extractor for Excel

## Overview
This notebook extracts high-quality tables from PDF files and formats them for Excel. Perfect for processing OCR-ed documents with structured data.

## How to Use
1. Click **"Install Dependencies"** button once
2. Click **"Extract Tables from PDF"** button for each file
3. Copy the displayed table and paste into Excel
4. Use **"Clear Workspace"** button between files

---

In [5]:
#@title 🔧 Install Dependencies - Enhanced OCR Version (Run Once) { display-mode: "form" }
#@markdown Click Run to install all packages including OCR support. Takes 2-3 minutes.

import sys
import subprocess

def install_packages():
    print("🔧 Installing enhanced packages with OCR support...")
    print("⏳ This may take 2-3 minutes...")

    # Install system dependencies
    print("📦 Installing system packages...")
    subprocess.run(['apt-get', 'update', '-qq'], capture_output=True)
    subprocess.run(['apt-get', 'install', '-y',
                   'ghostscript',
                   'python3-tk',
                   'tesseract-ocr',           # OCR engine
                   'tesseract-ocr-eng',       # English language data
                   'poppler-utils',           # PDF utilities
                   'ffmpeg',                  # Media processing
                   'libsm6',                  # OpenCV dependencies
                   'libxext6',
                   '-qq'], capture_output=True)

    # Install Python packages
    print("🐍 Installing Python packages...")
    packages = [
        'camelot-py[cv]',
        'tabula-py',
        'pandas',
        'openpyxl',
        'pdfplumber',              # Better OCR PDF handling
        'pymupdf',                 # PDF to image conversion
        'pytesseract',             # OCR Python wrapper
        'opencv-python-headless',  # Image preprocessing
        'Pillow',                  # Image handling
        'numpy'                    # Array operations
    ]

    subprocess.run([sys.executable, '-m', 'pip', 'install'] + packages + ['-q'],
                  capture_output=True)

    # Set display options
    import pandas as pd
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', 10)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 50)

    # Verify installations
    print("\n🔍 Verifying installations...")
    try:
        import camelot
        import tabula
        import pdfplumber
        import fitz
        import cv2
        import pytesseract
        print("✅ All packages installed successfully!")

        # Check Tesseract
        tesseract_version = subprocess.run(['tesseract', '--version'],
                                         capture_output=True, text=True)
        if tesseract_version.returncode == 0:
            print("✅ Tesseract OCR ready")

    except ImportError as e:
        print(f"⚠️  Some packages failed: {e}")
        print("Try running the cell again")

    print("\n👇 Ready! Scroll down to extract tables from PDFs")

install_packages()

🔧 Installing enhanced packages with OCR support...
⏳ This may take 2-3 minutes...
📦 Installing system packages...
🐍 Installing Python packages...

🔍 Verifying installations...
✅ All packages installed successfully!
✅ Tesseract OCR ready

👇 Ready! Scroll down to extract tables from PDFs


In [7]:
#@title 📊 Enhanced PDF Table Extractor (OCR-Ready) { display-mode: "form" }
#@markdown Advanced extraction with OCR support and multiple fallback methods

import camelot
import tabula
import pandas as pd
import pdfplumber
import fitz  # PyMuPDF
import cv2
import numpy as np
from PIL import Image
import pytesseract
from google.colab import files
import tempfile
import os
import io

# Install required packages (run once)
# !pip install camelot-py[cv] tabula-py pdfplumber pymupdf pytesseract opencv-python-headless

def preprocess_image_for_ocr(image):
    """Enhance image quality for better OCR"""
    # Convert to grayscale
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)

    # Denoise
    denoised = cv2.fastNlMeansDenoising(gray)

    # Adaptive thresholding for better text detection
    thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)

    # Deskew if needed
    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = 90 + angle
    if abs(angle) > 0.5:
        (h, w) = thresh.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        thresh = cv2.warpAffine(thresh, M, (w, h),
                               flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return thresh

def extract_with_pdfplumber(file_path):
    """Extract using pdfplumber - good for OCR-ed PDFs"""
    tables_data = []
    try:
        with pdfplumber.open(file_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 1):
                tables = page.extract_tables(table_settings={
                    "vertical_strategy": "lines_strict",
                    "horizontal_strategy": "lines_strict",
                    "explicit_vertical_lines": [],
                    "explicit_horizontal_lines": [],
                    "snap_tolerance": 3,
                    "join_tolerance": 3,
                    "edge_min_length": 5,
                    "min_words_vertical": 1,
                    "min_words_horizontal": 1,
                    "text_tolerance": 3,
                    "text_x_tolerance": None,
                    "text_y_tolerance": None,
                })

                for i, table in enumerate(tables):
                    if table:
                        df = pd.DataFrame(table)
                        # Clean empty rows/columns
                        df = df.dropna(how='all').dropna(axis=1, how='all')
                        if not df.empty:
                            tables_data.append({
                                'dataframe': df,
                                'page': page_num,
                                'method': 'pdfplumber'
                            })
    except Exception as e:
        print(f"❌ pdfplumber error: {str(e)}")
    return tables_data

def extract_with_ocr_pymupdf(file_path):
    """Direct OCR extraction using PyMuPDF and Tesseract"""
    tables_data = []
    try:
        pdf_document = fitz.open(file_path)

        for page_num, page in enumerate(pdf_document, 1):
            # Convert page to image
            mat = fitz.Matrix(2, 2)  # 2x zoom for better OCR
            pix = page.get_pixmap(matrix=mat)
            img_data = pix.pil_tobytes(format="PNG")
            image = Image.open(io.BytesIO(img_data))

            # Preprocess image
            processed = preprocess_image_for_ocr(image)

            # OCR with table detection hints
            custom_config = r'--oem 3 --psm 6 -c preserve_interword_spaces=1'
            text = pytesseract.image_to_data(processed, output_type=pytesseract.Output.DICT,
                                           config=custom_config)

            # Group text by lines and detect table structure
            df = pd.DataFrame(text)
            df = df[df['conf'] > 30]  # Filter low confidence

            if not df.empty:
                # Simple table reconstruction based on position
                df['line_group'] = df['line_num']
                table_data = []

                for line_num in df['line_group'].unique():
                    line_data = df[df['line_group'] == line_num]
                    if len(line_data) > 1:  # Multiple words in line = potential table row
                        row = line_data.sort_values('left')['text'].tolist()
                        table_data.append(row)

                if len(table_data) > 1:  # At least 2 rows
                    result_df = pd.DataFrame(table_data)
                    tables_data.append({
                        'dataframe': result_df,
                        'page': page_num,
                        'method': 'OCR-PyMuPDF'
                    })

    except Exception as e:
        print(f"❌ OCR extraction error: {str(e)}")
    return tables_data

def extract_tables_enhanced(file_path):
    """Multi-method extraction with fallbacks"""
    all_tables = []

    # Method 1: Camelot with both lattice and stream
    print("🎯 Trying Camelot (lattice)...")
    try:
        tables = camelot.read_pdf(file_path, flavor='lattice', pages='all',
                                 line_scale=40, copy_text=['v', 'h'])
        for table in tables:
            if not table.df.empty:
                all_tables.append({
                    'dataframe': table.df,
                    'page': table.page,
                    'accuracy': table.accuracy,
                    'method': 'camelot-lattice'
                })
    except:
        pass

    if len(all_tables) == 0:
        print("🎯 Trying Camelot (stream)...")
        try:
            tables = camelot.read_pdf(file_path, flavor='stream', pages='all',
                                     edge_tol=50, row_tol=10, column_tol=10)
            for table in tables:
                if not table.df.empty:
                    all_tables.append({
                        'dataframe': table.df,
                        'page': table.page,
                        'accuracy': table.accuracy,
                        'method': 'camelot-stream'
                    })
        except:
            pass

    # Method 2: pdfplumber (good for OCR)
    if len(all_tables) < 3:  # Try if few tables found
        print("🎯 Trying pdfplumber...")
        plumber_tables = extract_with_pdfplumber(file_path)
        all_tables.extend(plumber_tables)

    # Method 3: Tabula with different strategies
    if len(all_tables) < 3:
        print("🎯 Trying Tabula...")
        try:
            # Try different extraction methods
            for lattice in [True, False]:
                tables = tabula.read_pdf(file_path, pages='all',
                                       multiple_tables=True, lattice=lattice)
                for i, table in enumerate(tables):
                    if not table.empty:
                        all_tables.append({
                            'dataframe': table,
                            'page': 'auto',
                            'accuracy': 0,
                            'method': f'tabula-{"lattice" if lattice else "stream"}'
                        })
        except:
            pass

    # Method 4: Direct OCR if still no good results
    if len(all_tables) < 2:
        print("🎯 Applying direct OCR extraction...")
        ocr_tables = extract_with_ocr_pymupdf(file_path)
        all_tables.extend(ocr_tables)

    return all_tables

def merge_similar_tables(tables, similarity_threshold=0.7):
    """Merge duplicate tables from different methods"""
    unique_tables = []

    for table in tables:
        is_duplicate = False
        df = table['dataframe']

        for unique in unique_tables:
            unique_df = unique['dataframe']
            # Check if tables are similar (same shape and content overlap)
            if df.shape == unique_df.shape:
                try:
                    overlap = (df == unique_df).sum().sum() / (df.shape[0] * df.shape[1])
                    if overlap > similarity_threshold:
                        is_duplicate = True
                        # Keep the one with higher accuracy
                        if table.get('accuracy', 0) > unique.get('accuracy', 0):
                            unique_tables.remove(unique)
                            unique_tables.append(table)
                        break
                except:
                    pass

        if not is_duplicate:
            unique_tables.append(table)

    return unique_tables

def format_for_excel_enhanced(tables):
    """Enhanced formatting with metadata"""
    if not tables:
        return None

    all_data = []

    for i, table_info in enumerate(tables):
        df = table_info['dataframe']
        if not df.empty:
            # Clean data
            df = df.replace('', np.nan)
            df = df.dropna(how='all').dropna(axis=1, how='all')

            if not df.empty:
                df_with_meta = df.copy()
                df_with_meta.insert(0, 'Table_#', i + 1)
                df_with_meta.insert(1, 'Page', table_info.get('page', 'N/A'))
                df_with_meta.insert(2, 'Method', table_info.get('method', 'N/A'))
                df_with_meta.insert(3, 'Quality', f"{table_info.get('accuracy', 0):.1f}%")
                all_data.append(df_with_meta)

    return pd.concat(all_data, ignore_index=True) if all_data else None

# Main processing
print("📁 Select your PDF file (OCR-ready):")
uploaded = files.upload()

if uploaded:
    for filename, file_content in uploaded.items():
        print(f"\n🔄 Processing: {filename}")

        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
            tmp_file.write(file_content)
            tmp_file_path = tmp_file.name

        try:
            # Extract tables with all methods
            all_tables = extract_tables_enhanced(tmp_file_path)

            # Remove duplicates
            unique_tables = merge_similar_tables(all_tables)

            if unique_tables:
                print(f"\n✅ Found {len(unique_tables)} unique tables")
                for i, table in enumerate(unique_tables):
                    print(f"   Table {i+1}: Page {table.get('page', 'N/A')}, "
                          f"Method: {table.get('method', 'N/A')}, "
                          f"Quality: {table.get('accuracy', 0):.1f}%")

                # Create Excel file
                excel_df = format_for_excel_enhanced(unique_tables)

                if excel_df is not None and not excel_df.empty:
                    base_name = filename.replace('.pdf', '')
                    excel_filename = f"{base_name}_extracted_tables.xlsx"

                    # Save with multiple sheets if needed
                    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                        # Main sheet with all tables
                        excel_df.to_excel(writer, sheet_name='All_Tables', index=False)

                        # Individual sheets for each table
                        for i, table in enumerate(unique_tables):
                            df = table['dataframe']
                            sheet_name = f"Table_{i+1}_P{table.get('page', 'X')}"[:31]
                            df.to_excel(writer, sheet_name=sheet_name, index=False)

                    files.download(excel_filename)
                    print(f"\n💾 Downloaded: {excel_filename}")
                    print("✅ Processing complete!")
                else:
                    print("❌ No valid tables extracted")
            else:
                print("❌ No tables found in PDF")
                print("💡 The PDF might need manual review or different OCR settings")

        finally:
            os.unlink(tmp_file_path)
else:
    print("❌ No file uploaded")

📁 Select your PDF file (Advanced OCR-ready):


Saving Sample D_EN 1.pdf to Sample D_EN 1 (1).pdf

🔄 Processing: Sample D_EN 1 (1).pdf
🔍 Using multi-pass extraction with merged cell detection...
🎯 Pass 1: Standard extraction...
🎯 Pass 2: Enhanced extraction with merged cell handling...
🎯 Pass 3: Advanced OCR extraction...

✅ Found 2 unique tables
   Table 1: Table_1_P1_camelot - 22 rows × 8 cols
   Table 2: Table_2_P1_OCR - 33 rows × 17 cols


/tmp/ipython-input-7-2121084215.py:409: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_normalized = df.replace('', np.nan).dropna(how='all').dropna(axis=1, how='all')
/tmp/ipython-input-7-2121084215.py:413: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  unique_normalized = unique_df.replace('', np.nan).dropna(how='all').dropna(axis=1, how='all')
/tmp/ipython-input-7-2121084215.py:474: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


💾 Downloaded: Sample D_EN 1 (1)_extracted_tables.xlsx
✅ Processing complete!


In [3]:
# @title 🗑️ Clear Input & Output Directory

import shutil
from pathlib import Path
import os

# Define the directories to be cleared
directories_to_clear = ["/content/sample_data"]

# Also clear uploaded files in /content (but preserve system folders)
content_dir = Path("/content")
system_folders = {".config", "sample_data", "__pycache__"}

# Warning message
print("⚠️ WARNING: This will delete all contents of the following directories:")
for directory in directories_to_clear:
    print(f"- {directory}")
print("- Uploaded files in /content (excluding system folders)")

#confirmation = input("Type 'YES' to confirm: ")

if True:
    # Clear specified directories
    for directory in directories_to_clear:
        dir_path = Path(directory)
        if dir_path.exists() and dir_path.is_dir():
            shutil.rmtree(dir_path)
            dir_path.mkdir(parents=True, exist_ok=True)
            print(f"✅ '{directory}' has been cleared.")
        else:
            print(f"The '{directory}' directory does not exist.")

    # Clear uploaded files from /content
    if content_dir.exists():
        for item in content_dir.iterdir():
            if item.name not in system_folders and item.is_file():
                item.unlink()
                print(f"✅ Removed uploaded file: {item.name}")
            elif item.is_dir() and item.name not in system_folders and item.name not in ["output", "sample_pdfs"]:
                shutil.rmtree(item)
                print(f"✅ Removed uploaded folder: {item.name}")

else:
    print("Operation cancelled. No files were deleted.")

⚠️ WARNING: This will delete all contents of the following directories:
- /content/sample_data
- Uploaded files in /content (excluding system folders)
✅ '/content/sample_data' has been cleared.
✅ Removed uploaded file: Sample D_EN 1.pdf
✅ Removed uploaded file: Sample D_EN 1_tables.xlsx
